## Data lake division
- Landing Zone (Raw)
- Clean zone (Transformations)
- Bussiness zone (Warehousing)

## Parquet File
- Apache Parquet is a popular column storage file format used by Hadoop systems, such as Pig, Spark, and Hive

## Singer
- The open-source stantard for writing scripts taht move data
- Tap (Extract data)
- Stream
    - Schema (metadata)
    - state (process metadata)
    - record (data)
- Target (loads data)

In [ ]:
# Import json
import json

database_address = {
  "host": "10.0.0.5",
  "port": 8456
}

# Open the configuration file in writable mode
with open("database_config.json", mode="w") as fh:
  # Serialize the object in this file handle
  json.dump(obj=database_address, fp=fh) 

In [ ]:
import singer

# Complete the JSON schema
schema = {'properties': {
    'brand': {'type': 'string'},
    'model': {'type': 'string'},
    'price': {'type': 'number'},
    'currency': {'type': 'string'},
    'quantity': {'type': 'number', 'minimum': 1},  
    'date': {'type': 'string', 'format': 'date'},
    'countrycode': {'type': 'string', 'pattern': "^[A-Z]{2}$"}, 
    'store_name': {'type': 'string'}}}

# Write the schema
singer.write_schema(stream_name="products", schema=schema, key_properties=[])

### API endpoit

In [ ]:
endpoint = "http://localhost:5000"

# Fill in the correct API key
api_key = "scientist007"

# Create the web API’s URL
authenticated_endpoint = "{}/{}".format(endpoint, api_key)

# Get the web API’s reply to the endpoint
api_response = requests.get(authenticated_endpoint).json()
pprint.pprint(api_response)

In [ ]:
# Use the convenience function to query the API
tesco_items = retrieve_products("Tesco")

singer.write_schema(stream_name="products", schema=schema,
                    key_properties=[])

# Write a single record to the stream, that adheres to the schema
singer.write_record(stream_name="products", 
                    record={**tesco_items[0], "store_name": "Tesco"})

for shop in requests.get(SHOPS_URL).json()["shops"]:
    # Write all of the records that you retrieve from the API
    singer.write_records(
      stream_name="products", # Use the same stream name that you used in the schema
      records=({**item, "store_name": shop}
               for item in retrieve_products(shop))
    )

In [ ]:
#tap_marketing_api.py
import requests
import singer

api_netloc = "localhost:5000"
api_key = "scientist007"
shops_template = f"http://{api_netloc}/{api_key}/diaper/api/v1.0/shops"
items_template = f"http://{api_netloc}/{api_key}/diaper/api/v1.0/items/"

# Complete the JSON schema
schema = {'properties': {
    'brand': {'type': 'string'},
    'model': {'type': 'string'},
    'price': {'type': 'number'},
    'currency': {'type': 'string'},
    'quantity': {'type': 'integer', 'minimum': 1},
    'date': {'type': 'string', "format": "date"},
    'countrycode': {'type': 'string', 'pattern': "^[A-Z]{2}$"},
    'store_name': {'type': 'string'}},
    '$schema': 'http://json-schema.org/draft-07/schema#'
}

# Write the schema to stdout.
singer.write_schema(stream_name='products', schema=schema, key_properties=[])


# Return the set of items scraped from a specific store as a list
def retrieve_store_items(store_name, items_endpoint=items_template):
    return requests.get(f"{items_endpoint}{store_name}").json()["items"]


def main():
    for shop in requests.get(shops_template).json()["shops"]:
        singer.write_records(stream_name='products',
                             # Add the name of the store to every record.
                             records=({'store_name': shop, **item}
                                      for item in retrieve_store_items(shop)))


if __name__ == "__main__":
    main()


In [ ]:
# tap-marketing-api | target-csv --config ingest/data_lake.conf